In [1]:
# 라이브러리 설치
!pip install langchain \
    langchain-community \
    langchain-pinecone \
    transformers \
    peft \
    torch \
    accelerate \
    streamlit \
    pinecone-client \
    sentence-transformers \
    fastapi \
    uvicorn \
    pyngrok \
    nest-asyncio \
    bitsandbytes \
    pypdf

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 

In [2]:
import os
import torch
import requests
import uvicorn
import nest_asyncio
import streamlit as st
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.llms.base import LLM
from peft import PeftModel, PeftConfig
from pinecone import Pinecone, ServerlessSpec
from typing import Optional, List, Mapping, Any
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_community.llms import HuggingFaceHub
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.embeddings import HuggingFaceEmbeddings


## 모델 불러오기

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8비트 양자화
    llm_int8_threshold=6.0  # 임계값 설정해야함
)

# 모델 이름 설정
model_name = "DLI-Lab/camel"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # `pad_token` 설정

# 모델 로드 및 양자화
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

### DST 부분 추가

In [4]:
!git clone https://github.com/ToLightUpTheSky/ParsingDST.git
%cd ParsingDST/ParsingDST/data
!pip install .

Cloning into 'ParsingDST'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 92 (delta 20), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 28.20 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/ParsingDST/ParsingDST/data
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [5]:
# MultiWOZ 2.1과 2.4 데이터셋 다운로드 및 전처리
!python create_data.py --main_dir mwz21 --mwz_ver 2.1 --target_path mwz2.1
!python create_data.py --main_dir mwz24 --mwz_ver 2.4 --target_path mwz2.4

Create WOZ-like dialogues. Get yourself a coffee, this might take a while.
Divide dialogues...
# of dialogues: Train 8420, Val 1000, Test 999
Create WOZ-like dialogues. Get yourself a coffee, this might take a while.
Divide dialogues...
# of dialogues: Train 8420, Val 1000, Test 999


In [6]:
class EmotionStateTracker:
    def __init__(self, ontology_path: str, model_path: str = None):
        # 온톨로지 및 모델 초기화
        with open(ontology_path, 'r') as f:
            self.ontology = json.load(f)
        self.model_path = model_path

    def extract_emotion(self, user_utterance: str, context: list) -> dict:
        # 사용자 발화 및 컨텍스트로부터 감정을 추출하는 함수
        # DST 모델의 감정 추출 로직을 구현했다
        prompt = f"User utterance: {user_utterance}\nContext: {context}\nExtract emotion states:"
        # 여기에 ParsingDST의 chat_reply 호출 또는 유사 로직 삽입
        emotion_state = {"emotion": "example_emotion"}
        return emotion_state

## Agent 정의

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모든 Agent에 다 양자화 해줘야 한다.
class Agent:
    def __init__(self, model_name: str, max_length: int = 512, device: str = "cuda"):
        # 8비트 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=6.0
        )

        # 모델 및 토크나이저 초기화
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token  # pad_token 설정. 여기도 안 하면 나중에 오류남
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto"
        ).to(device)
        self.max_length = max_length
        self.device = device

    def generate(self, prompt: str) -> str:
        # 입력 텍스트를 토큰화
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=self.max_length, padding=True).to(self.device)

        # 모델로 텍스트 생성
        outputs = self.model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=self.max_length,
            do_sample=True,
            pad_token_id=self.tokenizer.pad_token_id
        )

        # 결과 디코딩
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

class CBTAgent:
    def __init__(self, model_name: str, prompt_template: str, ontology_path: str, max_length: int = 512, device: str = "cuda"):
        # 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,  # 8비트 양자화 활성화
            llm_int8_threshold=6.0  # 임계값 설정
        )

        # 모델 및 토크나이저 초기화
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token  # `pad_token` 설정
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,  # 8비트 양자화 적용
            device_map="auto"  # GPU 자동 할당
        ).to(device)
        self.model.config.pad_token_id = self.tokenizer.pad_token_id  # 모델 설정 일치
        self.prompt_template = prompt_template
        self.max_length = max_length
        self.device = device

        # EmotionStateTracker 초기화
        self.emotion_tracker = EmotionStateTracker(ontology_path)


    def generate(self, client_information, reason, history):
        # 상담 기록(history) 포맷팅
        history_text = '\n'.join(
            [
                f"{message['role'].capitalize()}: {message['message']}"
                for message in history
            ]
        )

        last_user_utterance = history[-1]["message"] if history else ""

        # 감정 추출
        emotion_state = self.emotion_tracker.extract_emotion(
            user_utterance=last_user_utterance,
            context=history_text
        )

        # 프롬프트 생성
        prompt = self.prompt_template.format(
            client_information=client_information,
            reason_counseling=reason,
            history=history_text,
            emotion_state=json.dumps(emotion_state)  # 감정 상태 추가
        )

        # 입력 텍스트를 토큰화
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=self.max_length, padding=True).to(self.device)

        # 모델로 응답 생성
        outputs = self.model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=self.max_length,
            do_sample=True,
            pad_token_id=self.tokenizer.pad_token_id
        )

        # 결과 디코딩
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # CBT 기법과 상담 계획 추출
        try:
            cbt_technique = response.split("Counseling")[0].strip()
        except:
            cbt_technique = None

        try:
            cbt_plan = response.split("Counseling planning:\n")[1].split("\nCBT")[0].strip()
        except:
            cbt_plan = None

        return cbt_technique, cbt_plan

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

class CounsleorAgent:
    def __init__(self, model_name: str, prompt_template: str, cbt_plan: str, max_length: int = 512, device: str = "cuda"):
        # 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,  # 8비트 양자화 활성화
            llm_int8_threshold=6.0  # 임계값 설정
        )

        # 모델 및 토크나이저 초기화
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token  # `pad_token` 설정

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,  # 8비트 양자화 적용
            device_map="auto"  # GPU 자동 할당
        ).to(device)
        self.model.config.pad_token_id = self.tokenizer.pad_token_id  # 모델 설정 일치

        self.cbt_plan = cbt_plan
        self.prompt_template = prompt_template
        self.max_length = max_length
        self.device = device

    def generate(self, client_information, reason, history):
        # 상담 기록(history) 포맷팅
        history_text = '\n'.join(
            [
                f"{message['role'].capitalize()}: {message['message']}"
                for message in history
            ]
        )

        # 프롬프트 생성
        prompt = self.prompt_template.format(
            client_information=client_information,
            reason_counseling=reason,
            cbt_plan=self.cbt_plan,
            history=history_text,
        )

        # 입력 텍스트를 토큰화
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=self.max_length, padding=True).to(self.device)

        # 모델로 응답 생성
        outputs = self.model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=self.max_length,
            do_sample=True,
            pad_token_id=self.tokenizer.pad_token_id
        )

        # 결과 디코딩
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 상담자 응답 추출
        try:
            counselor_response = response.split("Counselor:")[-1].strip()
        except IndexError:
            counselor_response = response.strip()

        return counselor_response


### 프롬프트

In [10]:
RESPONSE_PROMPT = """
<|start_header_id|>system<|end_header_id|>
You are a counselor specializing in psychological counseling. Your task is to use the provided client information, counseling plan, and emotion state to generate a response.
<|eot_id|><|start_header_id|>user<|end_header_id|>
Client Information:
{client_information}
Reason for Counseling:
{reason_counseling}
Emotion State:
{emotion_state}
Counseling Dialogue:
{history}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


In [11]:
CBT_PLAN_PROMPT="""<|start_header_id|>system<|end_header_id|>

You are a counselor specializing in CBT techniques. Your task is to use the provided client information, and dialogue to generate an appropriate CBT technique and a detailed counseling plan.<|eot_id|><|start_header_id|>user<|end_header_id|>

Types of CBT Techniques:
Efficiency Evaluation, Pie Chart Technique, Alternative Perspective, Decatastrophizing, Pros and Cons Analysis, Evidence-Based Questioning, Reality Testing, Continuum Technique, Changing Rules to Wishes, Behavior Experiment, Problem-Solving Skills Training, Systematic Exposure

Client Information:
{client_information}

Reason for seeking counseling:
{reason_counseling}

Counseling Dialogue:
{history}

Choose an appropriate CBT technique and create a counseling plan based on that technique.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

# 입력 값 유효성 검사 함수
def validate_input(prompt, default_value):
    user_input = input(prompt).strip()
    return user_input if user_input else default_value

# Client 정보 수집
def collect_info(name, age, gender, occupation, education, marital_status, family_details, reason):
    CLIENT_INFO = f"""Name: {name}
Age: {age}
Gender: {gender}
Occupation: {occupation}
Education: {education}
Marital Status: {marital_status}
Family Details: {family_details}"""

    REASON_FOR_COUNSELING = reason
    HISTORY_INIT = f"Counselor: Hi {name}, it's nice to meet you. How can I assist you today?\nClient: "

    return CLIENT_INFO, REASON_FOR_COUNSELING, HISTORY_INIT

# 상담 세션 시작
def start_demo(model_name, intake_form, reason, history_init, max_turns=20, device="cuda"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )

    print("Welcome to the Multi-Turn ClientAgent Demo!\n")
    print(f"[Intake Form]")
    print(intake_form)
    print("Type 'exit' to quit the demo.\n")

    print("====== Counseling Session ======\n")
    print(f"Counselor: Hi, it's nice to meet you. How can I assist you today?")

    history = []
    num_turn = 0

    while num_turn < max_turns:
        user_input = input("You (Client): ")
        if user_input.lower() == "exit":
            print("\n====== Exiting the demo. Goodbye! ======\n")
            break

        history.append({"role": "Client", "message": user_input})
        prompt = f"""Client Information:
{intake_form}

Reason for Counseling:
{reason}

Counseling Dialogue:
{history}

Counselor:"""

        try:
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)
            outputs = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=512,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )
            counselor_response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Counselor:")[-1].strip()
        except Exception as e:
            counselor_response = f"Error: {e}"

        print(f"Counselor: {counselor_response}")
        history.append({"role": "Counselor", "message": counselor_response})
        num_turn += 1

    print("Demo completed.")
    return history

# 사용자 입력
name = validate_input("Let's begin the pre-counseling session. What is your name? ", "Anonymous")
age = validate_input("How old are you? ", "Unknown")
gender = validate_input("What is your gender? (e.g., Male, Female): ", "Not specified")
occupation = validate_input("What is your occupation? ", "Unemployed")
education = validate_input("What is your highest level of education? (e.g., College Graduate): ", "Unknown")
marital_status = validate_input("What is your marital status? (e.g., Single, Married): ", "Single")
family_details = validate_input("Can you briefly describe your family situation? (e.g., Lives alone): ", "Not specified")
reason = validate_input("What brings you here for counseling? Please explain briefly: ", "General inquiry")

# Client 정보 생성
CLIENT_INFO, REASON_FOR_COUNSELING, HISTORY_INIT = collect_info(name, age, gender, occupation, education, marital_status, family_details, reason)

# 모델 이름 설정
model_name = "DLI-Lab/camel"

# 상담 세션 실행
history = start_demo(model_name, CLIENT_INFO, REASON_FOR_COUNSELING, HISTORY_INIT)

# 상담 기록 출력
for message in history:
    print(f"{message['role']}: {message['message']}")

Let's begin the pre-counseling session. What is your name? kyungmin
How old are you? 24
What is your gender? (e.g., Male, Female): female
What is your occupation? student
What is your highest level of education? (e.g., College Graduate): college
What is your marital status? (e.g., Single, Married): single
Can you briefly describe your family situation? (e.g., Lives alone): lives alone
What brings you here for counseling? Please explain briefly: sad


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Welcome to the Multi-Turn ClientAgent Demo!

[Intake Form]
Name: kyungmin
Age: 24
Gender: female
Occupation: student
Education: college
Marital Status: single
Family Details: lives alone
Type 'exit' to quit the demo.

====== Counseling Session ======

Counselor: Hi, it's nice to meet you. How can I assist you today?


KeyboardInterrupt: Interrupted by user

# 상담 애플리케이션 Colab 배포


In [13]:
!pip install streamlit transformers torch bitsandbytes pyngrok
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 2s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴npm notice
npm notice New major version of npm available! 10.8.2 -> 11.0.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.0.0
npm notice To update run: npm install -g npm@11.0.0
npm notice
⠴

In [14]:
# 배포 코드 구현한 부분...
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

class CounselingApp:
    def __init__(self):
        if 'history' not in st.session_state:
            st.session_state.history = []
        if 'client_info' not in st.session_state:
            st.session_state.client_info = None
        if 'model' not in st.session_state:
            st.session_state.model = None
        if 'tokenizer' not in st.session_state:
            st.session_state.tokenizer = None

    def load_model(self):
        model_name = "DLI-Lab/camel"
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=6.0
        )

        st.session_state.tokenizer = AutoTokenizer.from_pretrained(model_name)
        st.session_state.tokenizer.pad_token = st.session_state.tokenizer.eos_token

        with st.spinner('Loading model... This might take a few minutes...'):
            st.session_state.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                device_map="auto"
            )

    def collect_client_info(self):
        st.title("Online Counseling Session")

        with st.form("client_info_form"):
            st.write("### Pre-counseling Information")
            name = st.text_input("Name", "Anonymous")
            age = st.text_input("Age", "Unknown")
            gender = st.selectbox("Gender", ["Not specified", "Male", "Female", "Other"])
            occupation = st.text_input("Occupation", "Not specified")
            education = st.text_input("Education Level", "Not specified")
            marital_status = st.selectbox("Marital Status",
                ["Single", "Married", "Divorced", "Widowed", "Not specified"])
            family_details = st.text_area("Family Situation", "Not specified")
            reason = st.text_area("Reason for seeking counseling", "General inquiry")

            submit_button = st.form_submit_button("Start Counseling")

            if submit_button:
                st.session_state.client_info = {
                    "info": f"""Name: {name}
Age: {age}
Gender: {gender}
Occupation: {occupation}
Education: {education}
Marital Status: {marital_status}
Family Details: {family_details}""",
                    "reason": reason
                }
                return True
        return False

    def generate_response(self, user_input):
        history_text = "\n".join([
            f"{msg['role']}: {msg['content']}"
            for msg in st.session_state.history
        ])

        prompt = f"""Client Information:
{st.session_state.client_info['info']}

Reason for Counseling:
{st.session_state.client_info['reason']}

Counseling Dialogue:
{history_text}
Client: {user_input}
Counselor:"""

        inputs = st.session_state.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to("cuda" if torch.cuda.is_available() else "cpu")

        outputs = st.session_state.model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=512,
            do_sample=True,
            pad_token_id=st.session_state.tokenizer.pad_token_id
        )

        response = st.session_state.tokenizer.decode(
            outputs[0],
            skip_special_tokens=True
        ).split("Counselor:")[-1].strip()

        return response

    def display_chat(self):
        for message in st.session_state.history:
            with st.chat_message(message["role"].lower()):
                st.write(message["content"])

    def run(self):
        if st.session_state.model is None:
            self.load_model()

        if st.session_state.client_info is None:
            if self.collect_client_info():
                st.rerun()
        else:
            st.title("Counseling Session")
            self.display_chat()

            if user_input := st.chat_input("Type your message here..."):
                with st.chat_message("client"):
                    st.write(user_input)
                st.session_state.history.append({"role": "Client", "content": user_input})

                with st.chat_message("counselor"):
                    response = self.generate_response(user_input)
                    st.write(response)
                st.session_state.history.append({"role": "Counselor", "content": response})

if __name__ == "__main__":
    app = CounselingApp()
    app.run()

Writing app.py


In [23]:
# 모든 프로세스 정리
!pkill -f ngrok
!pkill -f streamlit

# streamlit 실행
!streamlit run app.py &>/content/logs.txt &

# 잠시 대기하자
import time
time.sleep(5)

# ngrok 설정
from pyngrok import ngrok

# ngrok 인증 토큰 설정
!ngrok authtoken "여기에 토큰넣기"

# 모든 활성 터널 종료
ngrok.kill()

time.sleep(2)  # 추가 대기...

# 새로운 터널 생성
try:
    public_url = ngrok.connect(8501)
    print(f'애플리케이션 접속 URL: {public_url}')
except Exception as e:
    print(f"Error: {str(e)}")
    print("Please check https://dashboard.ngrok.com/agents to ensure no other sessions are running")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
애플리케이션 접속 URL: NgrokTunnel: "https://0369-34-125-172-59.ngrok-free.app" -> "http://localhost:8501"
